In [1]:
#!/usr/bin/python3

from redpitaya import *

for i in range(4):
    val = base.AIpinGetValue(i)
    print("Measured voltage on AI[%d] = {%f} V" % (i, val))

Measured voltage on AI[0] = {0.042735} V
Measured voltage on AI[1] = {0.070085} V
Measured voltage on AI[2] = {0.039316} V
Measured voltage on AI[3] = {0.042735} V
